In [ ]:
import os
import sys
import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer
import json

optimizer = StrategyOptimizer(root_path=root_path)
dfs = []
for study_name in ["xgridt_2024-11-22"]:
    trials_df = optimizer.get_study_trials_df(study_name)
    trials_df["study_name"] = study_name
    dfs.append(trials_df)
trials = pd.concat(dfs)
def preprocess_trials_df(trials_df):
    df = trials_df.copy()
    df.columns = [col.replace("params_", "") if col.startswith("params_") else col for col in df.columns]
    df["executors"] = df["executors"].apply(lambda x: json.loads(x))
    df["config"] = df["config"].apply(lambda x: json.loads(x))
    df["trading_pair"] = df["config"].apply(lambda x: x['trading_pair'])
    df["from_timestamp"] = df["executors"].apply(
        lambda x: pd.to_datetime(pd.Series(x["timestamp"].values()).min(), unit="s"))
    df["to_timestamp"] = df["executors"].apply(
        lambda x: pd.to_datetime(pd.Series(x["close_timestamp"].values()).max(), unit="s"))
    return df
trials = preprocess_trials_df(trials)

trials['from_timestamp'] = pd.to_datetime(trials['from_timestamp'])
trials['to_timestamp'] = pd.to_datetime(trials['to_timestamp'])
trials['days_diff'] = (trials['to_timestamp'] - trials['from_timestamp']).dt.total_seconds() / 86400
trials['executors_per_day'] = trials['total_executors'] / trials['days_diff']
trials['volume_per_day'] = trials['total_volume'] / trials['days_diff']

In [ ]:
len(trials)

In [ ]:
trials.drop_duplicates(subset=["config"], inplace=True)
len(trials)

In [ ]:
def get_best_trials(df, top_results_per_pair=3):
    final_df = pd.DataFrame()

    for trading_pair in df['trading_pair'].unique():
        trading_pair_df = df[(df['trading_pair'] == trading_pair) & (df['executors'])]
        # filtered = trading_pair_df[
        #     (trading_pair_df['executors_per_day'] >= trading_pair_df['executors_per_day'].median()) &
        #     (trading_pair_df['volume_per_day'] >= trading_pair_df['volume_per_day'].median()) &
        #     (trading_pair_df['sharpe_ratio'] >= trading_pair_df['sharpe_ratio'].median()) &
        #     (trading_pair_df['max_drawdown_pct'] <= trading_pair_df['max_drawdown_pct'].median())
        # ]
        filtered = trading_pair_df.copy()

        if filtered.empty: 
            continue

        normalized_columns = ['executors_per_day', 'volume_per_day', 'sharpe_ratio', 'max_drawdown_pct']
        for col in normalized_columns:
            filtered[col] = filtered[col] / filtered[col].max()

        filtered['score'] = (
            filtered['executors_per_day'] *
            filtered['volume_per_day'] *
            filtered['sharpe_ratio'] *
            (1 / filtered['max_drawdown_pct'])
        )
        filtered['score'] = filtered['score'] / filtered['score'].max()
        filtered = filtered.dropna().sort_values(by='score', ascending=False).head(top_results_per_pair)

        final_df = pd.concat([final_df, filtered[['study_name', 'config', 'trading_pair', 'max_drawdown_pct', 'executors_per_day', 'sharpe_ratio', 'volume_per_day', 'score']]])
    return final_df

# Call the function and pass the dataframe
best_trials = get_best_trials(trials, top_results_per_pair=3)
best_trials

In [ ]:
import json
from core.services.backend_api_client import BackendAPIClient

configs = []
for index, row in best_trials[best_trials["trading_pair"] == "PNUT-USDT"].iterrows():
    config = row["config"]
    config["natr_length"] = 100
    config["leverage"] = 50
    config["controller_type"] = "generic"
    config["id"] = row["study_name"] + "_" + row["trading_pair"] + "_" + str(index)
    config["prominence_pct_peaks"] = 0.05
    config["distance_between_peaks"] = 100
    config["executor_activation_bounds"] = 0.001
    config["general_activation_bounds"] = 0.002
    config["max_ranges_by_signal"] = 1
    config["min_spread_between_orders"] = 0.0008
    config["min_order_amount"] = 5.5
    config["max_open_orders"] = 5
    config["max_orders_per_batch"] = 1
    config["order_frequency"] = 7
    config["close_position_on_signal_change"] = True
    config["grid_update_interval"] = None
    config["take_profit_mode"] = "original"
    config["take_profit_step_multiplier"] = 1
    config["global_stop_loss"] = 0.1
    config["total_amount_quote"] = 500
    config.pop("cooldown_time")
    config.pop("stop_loss")
    config.pop("max_executors_per_side")
    config.pop("take_profit")
    config.pop("take_profit_order_type")
    config.pop("trailing_stop")
    config.pop("candles_config")
    configs.append(row["config"])

In [ ]:
from core.utils import dump_dict_to_yaml

dump_dict_to_yaml(configs[2], folder="/Users/dman/Documents/code/hummingbot/conf/controllers")

In [ ]:
configs = best_trials["config"].tolist()
for config in configs:
    config["total_amount_quote"] = 100
    config["id"] = config["id"].replace("_", "-")

print(configs)
config_ids = [config["id"] + ".yml" for config in configs]
print(config_ids)

In [ ]:
client = BackendAPIClient(
    host="localhost",
    port=8000,
)
for config in configs:
    await client.add_controller_config(config)


In [ ]:
await client.deploy_script_with_controllers(
        bot_name="test_1",
        controller_configs=config_ids,
        image_name="hummingbot/hummingbot:latest",
        max_global_drawdown=50,
        max_controller_drawdown=20,
    )